In [2]:
from datetime import datetime
from operator import itemgetter
from os import path

from astropy.table import Table
import numpy as np
from skynet.api.v20 import errors
from skynet.api.v20.methods import download, groups, obs, scopes

In [5]:
# define parameters to control the code below

data_directory = "Data"
start_time = "2024-08-26T21:10:00" # only download files after this time
# start_time = "2024-08-15 00:00:00" # only download files after this time
observation_criteria = {"groupId": groups.get("Majewski Research Group")["id"]}
exposure_criteria = observation_criteria

# scope_id = scopes.query(include="*", **{"name": "RRRT"})[0]["id"]

observation_criteria

{'groupId': 816}

In [6]:
# retrieve list of observations happening after start_time and meeting observation_criteria
# write to csv just in case it's useful

observation_fields = {
    "id": "Observation ID",
    "state": "Observation state",
    "priority": "Priority",
    "name": "Target Name",
    "user.username": "Username",
    "group.name": "Group",
    "raHours": "RA",
    "decDegs": "Dec",
    "currentTelescope.name": "Telescope",
    "teleOwner.name": "Scope Owner",
    "timeIn": "Submitted",
}

observation_results = obs.query(include=",".join(observation_fields.keys()), after=start_time, **observation_criteria)

observations = Table()
for field, col_name in observation_fields.items():
    field_parts = field.split(".")
    match len(field_parts):
        case 1:
            observations[col_name] = list(map(itemgetter(field), observation_results))
        case 2:
            field_dicts = list(map(itemgetter(field_parts[0]), observation_results))
            observations[col_name] = [field_dict[field_parts[1]] if field_dict is not None else "" for field_dict in field_dicts]
observations.write("skynet_observations.csv", overwrite=True)

observations

Observation ID,Observation state,Priority,Target Name,Username,Group,RA,Dec,Telescope,Scope Owner,Submitted
int32,str8,int32,str13,str7,str23,float64,float64,str1,str3,str19
11601694,canceled,6,tic 278465736,gnh3pgz,Majewski Research Group,21.65336111111111,51.34663888888889,,UVA,2024-08-26T21:14:18
11608344,active,4,tic 344541836,gnh3pgz,Majewski Research Group,21.19002777777778,57.62038888888889,,UVA,2024-08-28T17:47:17


In [7]:
# retrieve list of images happening after start_time and meeting exposure_criteria
# write to csv just in case it's useful

exposure_fields = {
    "exps.obsId": "Observation ID",
    "exps.expNum": "Exposure Num",
    "exps.imageId": "Image ID",
    "exps.obs.name": "Name",
    "exps.type": "Type",
    "exps.filterName": "Filter Name",
    "exps.timeTaken": "Time Taken",
    "exps.state": "Image State",
    "exps.expLengthUsed": "Exposure",
    "exps.binningUsed": "Binning",
    "exps.gain": "Gain",
    "exps.camera.name": "Camera",
}

exposure_results = obs.query(include=",".join(exposure_fields.keys()), after=start_time, **exposure_criteria)

exp_structs = [image for result in exposure_results for image in result["exps"]]
images = Table()
for field, col_name in exposure_fields.items():
    field_parts = field.split(".")
    match len(field_parts):
        case 2:
            images[col_name] = list(map(itemgetter(field_parts[1]), exp_structs))
        case 3:
            field_dicts = list(map(itemgetter(field_parts[1]), exp_structs))
            images[col_name] = [field_dict[field_parts[2]] if field_dict is not None else "" for field_dict in field_dicts]
images.write("skynet_images.csv", overwrite=True)

images

Observation ID,Exposure Num,Image ID,Name,Type,Filter Name,Time Taken,Image State,Exposure,Binning,Gain,Camera
int32,int32,str9,str13,str5,str1,object,str8,object,int32,object,str40
11601694,0,r43810130,tic 278465736,light,V,2024-08-28T08:48:53,archived,10.0,2,1.32256,UVA SBIG STX-16803 with FLI Filter Wheel
11601694,1,r43810131,tic 278465736,light,V,2024-08-28T08:49:11,archived,10.0,2,1.32256,UVA SBIG STX-16803 with FLI Filter Wheel
11601694,2,r43810132,tic 278465736,light,V,2024-08-28T08:49:29,archived,10.0,2,1.32256,UVA SBIG STX-16803 with FLI Filter Wheel
11601694,3,r43810133,tic 278465736,light,V,2024-08-28T08:49:47,archived,10.0,2,1.32256,UVA SBIG STX-16803 with FLI Filter Wheel
11601694,4,r43810134,tic 278465736,light,V,2024-08-28T08:50:05,archived,10.0,2,1.32256,UVA SBIG STX-16803 with FLI Filter Wheel
11601694,5,r43810135,tic 278465736,light,V,2024-08-28T08:50:24,archived,10.0,2,1.32256,UVA SBIG STX-16803 with FLI Filter Wheel
11601694,6,r43810136,tic 278465736,light,V,2024-08-28T08:50:41,archived,10.0,2,1.32256,UVA SBIG STX-16803 with FLI Filter Wheel
11601694,7,r43810137,tic 278465736,light,V,2024-08-28T08:51:00,archived,10.0,2,1.32256,UVA SBIG STX-16803 with FLI Filter Wheel
11601694,8,r43810138,tic 278465736,light,V,2024-08-28T08:51:18,archived,10.0,2,1.32256,UVA SBIG STX-16803 with FLI Filter Wheel


In [8]:
# do the actual download of the identified files
# skip files that already exist

for observation in observations:
    obs_id = observation[observation_fields["id"]]
    target = observation[observation_fields["name"]].strip().replace(" ", "_")
    filename = f"{data_directory}/{obs_id}-{target}.zip"
    if path.isfile(filename):
        print(f"Observation {obs_id:10d} already downloaded")
        continue
    print(f"Downloading observation {obs_id:10d}...", end=" ")
    start = datetime.now()
    try:
        _, data = download.fits(**{"obs": obs_id})
    except errors.BadRequestError:
        print("skipped - got bad request")
        continue
    filesize = len(data)
    duration = datetime.now() - start
    if filesize > 0:
        with open(filename, "wb") as f:
            f.write(data)
        print(f"got {filesize/1024/1024:7.1f}MB at {filesize / duration.seconds * 8 / 1024 / 1024:5.2f}Mbps saved to {filename}")
print("---Download Complete---")

Observation   11601694 already downloaded
---Download Complete---
